In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Input
from tensorflow.keras.models import Model, Sequential, save_model, load_model

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.efficientnet import EfficientNetB0

from keras.utils import np_utils

import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import os

import matplotlib.pyplot as plt

import pickle

In [ ]:
def plot_history(history, yrange):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    plt.legend(['train', 'test'], loc='upper left')
    
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
fh = open(DATASET_NAME, 'rb')
dataset = pickle.load(fh)

In [ ]:
input_shape2 = (224, 224, 1)

In [ ]:
random.shuffle(dataset)
train = dataset[:125]
val = dataset[125:165]
test = dataset[165:]

X_train, y_train = zip(*train)
X_val, y_val = zip(*val)
X_test, y_test = zip(*test)

X_train = np.array([x.reshape(INPUT_SHAPE) for x in X_train])
X_val = np.array([x.reshape(INPUT_SHAPE) for x in X_val])
X_test = np.array([x.reshape(INPUT_SHAPE) for x in X_test])

y_train = np.array(np_utils.to_categorical(y_train, 2))
y_val = np.array(np_utils.to_categorical(y_val, 2))
y_test = np.array(np_utils.to_categorical(y_test, 2))

In [ ]:
dropout_rates = [0.5, 0.6, 0.7, 0.8]
activations = ['relu', 'sigmoid']

In [ ]:
baseModel = MODEL_TYPE(weights=None, include_top=False, input_shape=INPUT_SHAPE)

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)

headModel = Dense(512, activation=ACTIVATION)(headModel)
headModel = Dropout(DROPOUT_RATE)(headModel)

headModel = Dense(512, activation=ACTIVATION)(headModel)
headModel = Dropout(DROPOUT_RATE)(headModel)

headModel = Dense(512, activation=ACTIVATION)(headModel)
headModel = Dropout(DROPOUT_RATE)(headModel)

headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer="Adam", 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(x=X_train,
          y=y_train,
          epochs=14,
          batch_size=128,
          validation_data=(X_val, y_val))

In [ ]:
score = model.evaluate(x=X_test,
                     y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plot_history(history, (0.4, 0.6))

In [ ]:
save_model(model, 
           NAME_OF_MODEL, 
           overwrite=True, 
           include_optimizer=True, 
           save_format=None, 
           signatures=None, options=None, save_traces=True)